<a href="https://colab.research.google.com/github/steciow/Colaborador/blob/main/Vacunas_COVID_DA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PROCESAMIENTO DE BASE SOBRE VACUNACION EN ARGENTINA**
**Vacunas en Argentina**

Fuente:

Vacunas contra COVID-19. Dosis Aplicadas en la República Argentina - Registro desagregado.
Se va a leer la tabla directamente de ese repositorio y se elaborarán tablas resumen.

## **Importar librerías**

In [ ]:
# importamos de librerías que se utilizarán para trabajar las tablas y le otorgamos un alias
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import plotly.express as px

In [ ]:
#para ver mejor el contenido de las tablas, indico el máximo de filas a visualizar en 400...
#pd.set_option('display.max_rows',800)

## **Levantar bases**

###**Base de datos vacunación**

In [ ]:
#levantar desde enlace web
url='https://sisa.msal.gov.ar/datos/descargas/covid-19/files/datos_nomivac_covid19.csv'
dfvacc1= pd.read_csv(url)

### **Bases auxiliares (clasificación y descripciones de distritos)**

In [ ]:
#levantar desde enlace web
url = 'https://raw.githubusercontent.com/steciow/municipios_regiones_PBA/main/info_mun.csv'
dfaux = pd.read_csv(url)

In [ ]:
url='https://raw.githubusercontent.com/steciow/jurisdiccionesARG/main/info_jur.csv'
dfaux2=pd.read_csv(url)

## **Explorando las bases**

In [ ]:
#para verificar el tamaño del listado de municipios con la agrupación en regiones
dfaux.shape

(135, 19)

In [ ]:
dfaux2.shape

(24, 16)

In [ ]:
dfaux.head(135)

,id_Mun,id_Prov_Mun,Municipio_PBA,Región,AMBA,Apertura_PBA_UNGS,Apertura_PBA_UNDAV,"lat,long",pob_mun_2015,pob_mun_2016,pob_mun_2017,pob_mun_2018,pob_mun_2019,pob_mun_2020,pob_mun_2021,pob_mun_2022,pob_mun_2023,pob_mun_2024,pob_mun_2025
0,7,6-7,Adolfo Alsina,Región I,Interior PBA,Interior PBA,Interior PBA,"-37.179483264541,-62.7599903900444",17356,17382,17407,17433,17458,17483,17507,17532,17556,17580,17603
1,14,6-14,Adolfo Gonzáles Chaves,Región I,Interior PBA,Interior PBA,Interior PBA,"-38.0323707296254,-60.0993919272956",12040,12016,11992,11968,11944,11922,11899,11877,11855,11832,11811
2,56,6-56,Bahía Blanca,Región I,Interior PBA,Interior PBA,Interior PBA,"-38.7227702774263,-62.2727419997435",307250,307836,308413,308982,309544,310095,310639,311175,311704,312224,312738
3,182,6-182,Coronel de Marina L. Rosales,Región I,Interior PBA,Interior PBA,Interior PBA,"-38.8781733536173,-62.0794804610544",63124,63203,63281,63358,63433,63508,63581,63654,63725,63795,63864
4,189,6-189,Coronel Dorrego,Región I,Interior PBA,Interior PBA,Interior PBA,"-38.7187998749142,-61.2883602471277",15589,15511,15435,15359,15285,15212,15141,15070,15000,14931,14864
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,648,6-648,Presidente Perón,Región XI,AMBA,Tercer cordón,Tercer cordón,"-34.9179865320528,-58.3821570397156",93960,96420,98847,101239,103597,105918,108203,110457,112680,114868,117024
131,655,6-655,Punta Indio,Región XI,Interior PBA,Interior PBA,Interior PBA,"-35.3874096777144,-57.338171581753",10282,10343,10404,10463,10521,10579,10635,10692,10747,10802,10855
132,778,6-778,San Vicente,Región XI,AMBA,Tercer cordón,Tercer cordón,"-35.0259018141174,-58.4224036114375",68634,70388,72118,73825,75506,77161,78791,80398,81983,83544,85082
133,812,6-812,Tordillo,Región XI,Interior PBA,Interior PBA,Interior PBA,"-36.5208605076063,-57.3249387125515",1794,1796,1799,1801,1804,1807,1809,1812,1814,1816,1818


In [ ]:
dfaux2.head(5)

,cod_jur,Jurisdiccion,Cod ISO 3166 alpha 2,Capital,Gobernador,pob_j_2015,pob_j_2016,pob_j_2017,pob_j_2018,pob_j_2019,pob_j_2020,pob_j_2021,pob_j_2022,pob_j_2023,pob_j_2024,pob_j_2025
0,2,Ciudad Autónoma de Buenos Aires,AR-C,NaN,Horacio Rodríguez Larreta,3054267,3059122,3063728,3068043,3072029,3075646,3078836,3081550,3083770,3085483,3086680
1,6,Buenos Aires,AR-B,La Plata,Axel Kicillof,16659931,16841135,17020012,17196396,17370144,17541141,17709598,17875743,18039509,18200851,18359753
2,10,Catamarca,AR-K,San Fernando del Valle de Catamarca,Raúl Jalil,396895,400678,404433,408152,411824,415438,418991,422476,425885,429208,432438
3,14,Córdoba,AR-X,Córdoba,Juan Schiaretti,3567654,3606540,3645321,3683937,3722332,3760450,3798261,3835738,3872830,3909489,3945677
4,18,Corrientes,AR-W,Corrientes,Gustavo Valdés,1070283,1080655,1090938,1101084,1111052,1120801,1130320,1139604,1148631,1157386,1165860


In [ ]:
dfvacc1.head(3)

,sexo,grupo_etario,jurisdiccion_residencia,jurisdiccion_residencia_id,depto_residencia,depto_residencia_id,jurisdiccion_aplicacion,jurisdiccion_aplicacion_id,depto_aplicacion,depto_aplicacion_id,fecha_aplicacion,vacuna,condicion_aplicacion,orden_dosis,lote_vacuna
0,F,40-49,Buenos Aires,6,Tandil,791,Buenos Aires,6,Tandil,791,2021-03-05,Sinopharm,Estratégico,1,2021010036
1,M,50-59,Buenos Aires,6,General Pueyrredón,357,Buenos Aires,6,General Pueyrredón,357,2021-04-05,Sputnik,Estratégico,1,I-220321
2,M,30-39,Entre Ríos,30,La Paz,70,Entre Ríos,30,La Paz,70,2021-01-12,Sputnik,Salud,1,486081120R


In [ ]:
dfvacc1.shape

(4771214, 15)

In [ ]:
dfvacc1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4771214 entries, 0 to 4771213
Data columns (total 15 columns):
 #   Column                      Dtype 
---  ------                      ----- 
 0   sexo                        object
 1   grupo_etario                object
 2   jurisdiccion_residencia     object
 3   jurisdiccion_residencia_id  int64 
 4   depto_residencia            object
 5   depto_residencia_id         int64 
 6   jurisdiccion_aplicacion     object
 7   jurisdiccion_aplicacion_id  int64 
 8   depto_aplicacion            object
 9   depto_aplicacion_id         int64 
 10  fecha_aplicacion            object
 11  vacuna                      object
 12  condicion_aplicacion        object
 13  orden_dosis                 int64 
 14  lote_vacuna                 object
dtypes: int64(5), object(10)
memory usage: 546.0+ MB


## **Elaboración de tablas**

###Eliminación de variables que no se usarán

In [ ]:
#soltar variables que no se usarán de la base con información de los municipios
dfaux.drop(columns=['id_Mun','pob_mun_2015','pob_mun_2016','pob_mun_2017','pob_mun_2018','pob_mun_2019','pob_mun_2022','pob_mun_2023','pob_mun_2024','pob_mun_2025'],axis=1,inplace=True)

In [ ]:
#soltar variables que no se usarán de la base con información de las provincias
dfaux2.drop(columns=['Gobernador','pob_j_2015','pob_j_2016','pob_j_2017','pob_j_2018','pob_j_2019','pob_j_2022','pob_j_2023','pob_j_2024','pob_j_2025'],axis=1,inplace=True)

### Agrupamiento y agregado de categorías en municipios

In [ ]:
#para homogeneizar el id de cada jurisdicción y poder vincular las tablas
dfvacc1.rename({'jurisdiccion_aplicacion_id':'cod_jur'},axis=1,inplace=True)
dfvacc1.rename({'depto_aplicacion_id':'id_Mun'},axis=1,inplace=True)
#para convertir la variable en una de texto. 
dfvacc1['cod_jur']=dfvacc1['cod_jur'].astype(str) 
dfvacc1['id_Mun']=dfvacc1['id_Mun'].astype(str) 
#genero un cod(concatenado PBA y Mun)= cod_provincia + cod_municipio
dfvacc1['id_Prov_Mun']=dfvacc1['cod_jur']+'-'+dfvacc1['id_Mun']

In [ ]:
#renombrar etiquetas
dfvacc1['orden_dosis'].replace({1:'Primera dosis', 
                    2:'Segunda dosis'
                    }, inplace=True)

In [ ]:
#esto es para convertir la variable en una de texto (se unirán tablas por variables que tienen que ser compatibles)
dfaux['id_Prov_Mun']=dfaux['id_Prov_Mun'].astype(str) 
dfaux2['cod_jur']=dfaux2['cod_jur'].astype(str) 

In [ ]:
#para obtener una nueva base mixta=que ahora tiene las regiones (y otras descripciones de municipios, para control)
#la base original se expande con las descripciones adicionales de los municipios de BA y la región sanitaria
df1p = pd.merge(dfvacc1, 
               dfaux, 
               on ='id_Prov_Mun', 
               how ='left')

In [ ]:
#para obtener una nueva base mixta=que ahora tiene las provincias (contiene los no especidificados con carga en esa jurisdicción)
dfcompleta = pd.merge(df1p, 
               dfaux2, 
               on ='cod_jur', 
               how ='left')

In [ ]:
dfcompleta.head(3)

,sexo,grupo_etario,jurisdiccion_residencia,jurisdiccion_residencia_id,depto_residencia,depto_residencia_id,jurisdiccion_aplicacion,cod_jur,depto_aplicacion,id_Mun,fecha_aplicacion,vacuna,condicion_aplicacion,orden_dosis,lote_vacuna,id_Prov_Mun,Municipio_PBA,Región,AMBA,Apertura_PBA_UNGS,Apertura_PBA_UNDAV,"lat,long",pob_mun_2020,pob_mun_2021,Jurisdiccion,Cod ISO 3166 alpha 2,Capital,pob_j_2020,pob_j_2021
0,F,40-49,Buenos Aires,6,Tandil,791,Buenos Aires,6,Tandil,791,2021-03-05,Sinopharm,Estratégico,Primera dosis,2021010036,6-791,Tandil,Región VIII,Interior PBA,Interior PBA,Interior PBA,"-37.3286954878065,-59.1368975553779",139300.0,140657.0,Buenos Aires,AR-B,La Plata,17541141,17709598
1,M,50-59,Buenos Aires,6,General Pueyrredón,357,Buenos Aires,6,General Pueyrredón,357,2021-04-05,Sputnik,Estratégico,Primera dosis,I-220321,6-357,General Pueyrredón,Región VIII,Interior PBA,Interior PBA,Interior PBA,"-38.0064494389904,-57.5438377446162",656456.0,659462.0,Buenos Aires,AR-B,La Plata,17541141,17709598
2,M,30-39,Entre Ríos,30,La Paz,70,Entre Ríos,30,La Paz,70,2021-01-12,Sputnik,Salud,Primera dosis,486081120R,30-70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Entre Ríos,AR-E,Paraná,1385961,1398510


In [ ]:
#creo las variables para contar primeras y segundas dosis
dfcompleta['1er dosis']=np.where(dfcompleta['orden_dosis']=='Primera dosis',1,0)
dfcompleta['2da dosis']=np.where(dfcompleta['orden_dosis']=='Segunda dosis',1,0)

### Elaboración de tablas resumen

In [ ]:
#tabla resumen final ordenada por fecha 
dfcompleta['fecha_aplicacion']=dfcompleta['fecha_aplicacion'].astype(str) #esto es para convertir la variable de fecha a una variable de texto
df1=dfcompleta.groupby(['fecha_aplicacion','condicion_aplicacion','Jurisdiccion','Cod ISO 3166 alpha 2','orden_dosis'],dropna=False).agg(
    aplicaciones=('orden_dosis','count'),
)

In [ ]:
#tabla resumen final por provincia, población y recuento de casos confirmados y fallecidos
df2=dfcompleta.groupby(['grupo_etario','vacuna','condicion_aplicacion','Jurisdiccion','Cod ISO 3166 alpha 2','pob_j_2021','Municipio_PBA','pob_mun_2021',	'Región',	'AMBA','lat,long','orden_dosis'],dropna=False).agg(
    Primeras_dosis=('1er dosis','sum'),
    Segundas_dosis=('2da dosis','sum'),
)

## **Guardado / descarga**

In [ ]:
#si se guarda en ubicación en drive como csv ('montar' e indicar ubicación en propio drive)
#esta es el resumen ordenado por fecha (verificar**)
df1.to_csv('/content/drive/MyDrive/Proyectos/COVID/vaccinedates.csv',header = True,index=True,sep=",")
df2.to_csv('/content/drive/MyDrive/Proyectos/COVID/vaccinedetail.csv',header = True,index=True,sep=",")
